In [ ]:
%matplotlib notebook
#%matplotlib widget
#%matplotlib inline

In [ ]:
# In Matplotlib 3.10+, this is needed to revert to a 3D view that fixes the Z axis ("roll"),
# which is vastly preferable for 3D plots of an air shower
#import matplotlib as mpl
#mpl.rcParams['axes3d.mouserotationstyle'] = 'azel'

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.coordinates import (
    SkyCoord, AltAz, EarthLocation, 
    spherical_to_cartesian, cartesian_to_spherical, 
)
from astropy.time import Time
from ctapipe.calib import CameraCalibrator
from ctapipe.coordinates import TelescopeFrame
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay

import cherentrace

In [ ]:
datadir = 'data/proton_20deg_90deg_cta-prod6-2147m-Paranal-lst-dark'
file = 'all_tables_5_TeV.simtel.zst'

source = EventSource(input_url=f'{datadir}/{file}')
calibrator = CameraCalibrator(subarray = source.subarray)

for event in source:
    calibrator(event)
    
    df_particles = cherentrace.get_particles(source, event)
    
    for telID,teldata in event.dl1.tel.items():
        geometry_cam = source.subarray.tels[telID].camera.geometry

        tel_frame = TelescopeFrame(telescope_pointing = SkyCoord(
          alt = event.pointing.tel[telID].altitude,
          az = event.pointing.tel[telID].azimuth,
          frame = AltAz(
            obstime = Time.now(),
            location = EarthLocation.of_site('Roque de los Muchachos'),
          ),
        ))
        geometry = geometry_cam.transform_to(tel_frame)
        
        df_photons = cherentrace.get_photons(source, event, telID, to_telescope_frame = True)
    
        true_image = event.simulation.tel[telID].true_image
        extract_image = teldata.image
        extract_time = teldata.peak_time
        break
    break

In [ ]:
df_photons

In [ ]:
df_particles

In [ ]:
muon_photons = df_photons[df_photons.is_muon]
muon_particles = df_particles[df_particles.is_muon]

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(9,4)

display1 = CameraDisplay(geometry, extract_image, ax = ax1, norm = 'log')
display1.add_colorbar(label = "Charge")

display2 = CameraDisplay(geometry, extract_time, ax = ax2, cmap = 'viridis')
display2.set_limits_minmax(*np.quantile(extract_time, [0.1, 0.99]))
display2.add_colorbar(label = "Arrival time")
fig.tight_layout()

hit = muon_photons.query("pixel_id >= 0")
for i,uid in enumerate(np.unique(hit.generation)):
    mask = (hit.generation == uid)
    c = f"C{i}"
    lon = hit[mask].x
    lat = hit[mask].y
    ax2.plot(lon, lat, '.', color = c, alpha = 0.7, label = f"G = {uid}")
    
ax2.legend(ncols = 2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
fig.set_size_inches(8,6)

for i,uid in enumerate(np.unique(muon_particles.particle_id)):
    mask = (
        (muon_particles.particle_id == uid) 
        & (abs(muon_particles.x) < 25_000) # hide outliers
        & (abs(muon_particles.y) < 25_000)
    )
    x = muon_particles[mask].x
    y = muon_particles[mask].y
    z = muon_particles[mask].z
    ax.plot(x, y, z, '.', color = f'C{i+1}', label = f"ID = {int(uid)}")
    
ax.update({
    'xlabel': "Northing (m)",
    'ylabel': "Westing (m)",
    'zlabel': "Altitude (m)",
    'title': "Muon positions",
})
fig.legend()

In [ ]:
generation_subset = None # Show all
# generation_subset = 54

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
fig.set_size_inches(8,6)

# Draw the approx ground scale of the selected telescope (assuming its at the array centre)
ground_z = source.subarray.reference_location.geodetic.height
mirror_r = np.sqrt(source.subarray.tels[telID].optics.mirror_area/np.pi).to_value('m')
array_x = -event.simulation.shower.core_x.to_value('m')
array_y = -event.simulation.shower.core_y.to_value('m')
ax.plot([array_x - mirror_r, array_x + mirror_r], array_y, ground_z, 'k-', label = "Mirror")
ax.plot(array_x, [array_y - mirror_r, array_y + mirror_r], ground_z, 'k-')

# Plot emissions positions of photons from muons
mask = (muon_photons.pixel_id >= 0)
if generation_subset:
    mask = mask & (muon_photons.generation == generation_subset)
x = muon_photons[mask].xem
y = muon_photons[mask].yem
z = muon_photons[mask].zem
alt = muon_photons[mask].alt
az = muon_photons[mask].az
pp = ax.plot(x, y, z, '.', label = "Photons from $\\mu$")
props = ax.properties()

# Plot direction vectors of emitted photons
len_path = 1000
cx,cy,cz = spherical_to_cartesian(len_path, alt.values*u.deg, -az.values*u.deg)
cx *= -1
cy *= -1
cz *= -1
## Really slow to plot because there are so many photons! Useful for debugging though
# ax.quiver3D(x, y, z, cx, cy, cz, alpha = 0.01, color = pp[0].get_color(),
#         arrow_length_ratio = 0.0, zorder = 999)

# Plot muon tracks
for i,uid in enumerate(np.unique(muon_particles.particle_id)):
    len_path = 1000 # metres. Set this to the spacing of the OBSLEVs
    scale_f = len_path/abs(muon_particles.cz) # Scale factor such that cz will be exactly len_path
    
    mask = (
        (muon_particles.particle_id == uid) 
        # remove long sideways tracks that cover more than 0.3 of the view window in one step
        & (abs(muon_particles.cx*scale_f) < 0.2*np.diff(np.asarray(props['xlim']))[0])
        & (abs(muon_particles.cy*scale_f) < 0.2*np.diff(np.asarray(props['ylim']))[0])
        # keep ground only
        # & (muon_particles.obs_level == muon_particles.obs_level.max())
    )
    if generation_subset:
        mask = mask & (muon_particles.generation == generation_subset)
        
    x = muon_particles[mask].x
    y = muon_particles[mask].y
    z = muon_particles[mask].z
    cx = muon_particles[mask].cx*scale_f[mask]
    cy = muon_particles[mask].cy*scale_f[mask]
    cz = muon_particles[mask].cz*scale_f[mask]
    # Make vectors point backwards to the originating direction, except for birth muons (70-80)
    if uid < 70 or uid > 90:
        cx *= -1
        cy *= -1
        cz *= -1
    
    pp = ax.plot(x, y, z, '.', label = f"ID = {int(uid)}")
    ax.quiver3D(x, y, z, cx, cy, cz, alpha = 0.5, color = pp[0].get_color(),
        arrow_length_ratio = 0.0, zorder = 999)

ax.update({
    'xlabel': "Northing (m)",
    'ylabel': "Westing (m)",
    'zlabel': "Altitude (m)",
    'xlim': props['xlim'],
    'ylim': props['ylim'],
    'zlim': props['zlim'],
    'title': "Muon positions and photon emission points"
})
fig.legend(loc = 'center right')